In [1]:
import os
import pickle
from tqdm.notebook import tqdm
import sentencepiece as spm

In [3]:
# source_folder = '/scratche/home/apoorv/EmbedKGQA/data/MetaQA'
source_folder = 'data/MetaQA_half'
! ls {source_folder}

entity_strings.txt	qa_train_2hop_half.txt	qa_valid_3hop.txt
qa_test_1hop.txt	qa_train_2hop.txt	test.txt
qa_test_2hop.txt	qa_train_3hop_half.txt	train.txt
qa_test_3hop.txt	qa_train_3hop.txt	valid.txt
qa_train_1hop_half.txt	qa_valid_1hop.txt
qa_train_1hop.txt	qa_valid_2hop.txt


In [4]:
def readLines(fname):
    f = open(fname)
    lines = []
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        lines.append(line)
    f.close()
    return lines

In [7]:
data = {}
hops = ['1', '2', '3']
for hop in hops:
    fname = os.path.join(source_folder, 'qa_train_'+ hop + 'hop.txt')
    data[hop] = readLines(fname)

In [8]:
data['1'][2]

'what films can be described by [occupation]\tRed Dawn|The Teahouse of the August Moon'

In [10]:
from unidecode import unidecode
def normalizeEntity(ent):
    ent = unidecode(ent)
    return ent

In [16]:
def separateEntity(question, replacement='NE'):
    split1 = question.split('[')
    lhs = split1[0]
    split2 = split1[1].split(']')
    entity = split2[0]
    rhs = split2[1]
    final = lhs + replacement + rhs
    return final, entity

def lineToKGCLines(line):
    line = line.split('\t')
    question, head_entity = separateEntity(line[0])
    head_entity = normalizeEntity(head_entity)
    base_line = 'predict answer: {0} | {1} |'.format(head_entity, question)
    answers = line[1].split('|') # multiple entities can be answer
    answers = [normalizeEntity(a) for a in answers]
    kgc_lines = []
    for a in answers:
        kgc_lines.append(base_line + '\t' + a)
    return kgc_lines

In [17]:
lineToKGCLines(data['1'][0])

['predict answer: ginger rogers | what movies are about NE |\tTop Hat',
 'predict answer: ginger rogers | what movies are about NE |\tKitty Foyle',
 'predict answer: ginger rogers | what movies are about NE |\tThe Barkleys of Broadway']

In [18]:
kgc_lines_1hop = []
for line in data['1']:
    kgc_lines_1hop.extend(lineToKGCLines(line))

In [22]:
kgc_lines_2hop = []
for line in data['2']:
    kgc_lines_2hop.extend(lineToKGCLines(line))

In [24]:
kgc_lines_3hop = []
for line in data['3']:
    kgc_lines_3hop.extend(lineToKGCLines(line))
len(kgc_lines_3hop)

1521495

In [23]:
len(kgc_lines_2hop)

739782

In [20]:
def writeLines(lines, fname):
    f = open(fname, 'w')
    for line in lines:
        f.write(line + '\n')
    f.close()

In [21]:
fname = 'data/MetaQA_half_1hop/train_base.txt'
writeLines(kgc_lines_1hop, fname)

In [25]:
fname = 'data/MetaQA_half_2hop/train_base.txt'
writeLines(kgc_lines_2hop, fname)

In [26]:
fname = 'data/MetaQA_half_3hop/train_base.txt'
writeLines(kgc_lines_3hop, fname)

In [15]:
# this cell is sanity check
source_folder = '/scratche/home/apoorv/EmbedKGQA/data/MetaQA'
! ls {source_folder}
fname = 'entities.dict'
entities = []
for line in open(os.path.join(source_folder, fname)):
    line = line.strip().split('\t')
    entities.append(line[0])
    
fname = 'relations.dict'
relations = []
for line in open(os.path.join(source_folder, fname)):
    line = line.strip().split('\t')
    relations.append(line[0])
count = 0
for e in entities:
    if e != unidecode(e):
        count +=1
count

entities.dict  relations.dict  test.txt  train.txt  valid.txt


959

In [28]:
len(entities), len(set(entities))

(123182, 123182)

In [33]:
len(entities_ud), len(set(entities_ud))

(123182, 123145)